### <p style="text-align: right;"> &#9989; Andrew </p>

# PHY480 Day 11

## In-class assignment: Linear algebra continued, eigenvalue problems

In this in-class assignment we examine the power iteration method that allows one to find the largest eigenvalue of a matrix and its corresponding eigenvector. We then examine the _inverse_ power iteration method with shifts that essentially allows one to find the eigenvalue closest to the shift and its corresponding eigenvector.

A simple algorithm for finding eigenvalues is the QR method which is based on QR decomposition. Once the eigenvalues are found, we can use the inverse power iteration method with shifts to find all eigenvectors.

The QR decomposition of $A$ is written as

$$
A = QR,
$$
where $Q$ is orthogonal, $Q^TQ=I$ and $R$ is upper triangular. (This can be generalized: if A is complex, then $Q$ is unitary, $Q^\dagger Q=I$.)


In [580]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


## Power iteration method

**Task 1.** A real matrix can potentially have complex eigenvalues and eigenvectors. To simplify the work that you are going to do in this notebook, we restrict the discussion to symmetric matrices $A^T=A$ that have real eigenvalues.

Generate a real symmetric matrix, i.e., generate a random matrix $A$ and then symmetrize it as

$$
A \leftarrow \frac{1}{2}(A+A^T).
$$

You will use this matrix in the exercises below. Once you have completed them, return to this point, generate matrices of different dimension and rerun all exercises, to check that everything works as intended for different input matrices.

Use `np.linalg.eig` to calculate the eigenvalues and eigenvectors, so you can use them for reference.


In [581]:
# YOUR CODE HERE
np.random.seed( 1 )

def random_symmetric_matrix(n):
    A = np.random.rand(n, n)
    return A + A.T

A = random_symmetric_matrix(5)
assert all(np.linalg.eig(A)[0].real == np.linalg.eig(A)[0])

**Task 2.** Code the power method that successively applies (symmetric) matrix $A$ to a starting vector (that can be chosen as, for instance, $[1\,1\,\dots1]$) and converges to the largest (in magnitude) eigenvalue and its eigenvector. Make sure to normalize the vector after each iteration. Return the eigenvalue and the eigenvector.


In [582]:
# power iteration method to find the largest eigenvalue and its eigenvector
# Input:
# A -- (square) matrix
# max_iter -- maximum number of iterations
# Output:
# largest eigenvalue, its eigenvector
def power_method( A, max_iter=100 ):
    '''
    currently no convergence evaluation
    '''

    # read off the dimension
    n = A.shape[0]

    b = np.ones(n)
    for _ in range(max_iter):
        bk = A@b
        b = bk / np.linalg.norm(bk)

    # Rayleigh Quotient
    eigenval = b@A@b/(b@b)

    return eigenval, b



In [583]:
# YOUR CODE HERE
print(
    power_method(A),
    '\n',
    max(np.linalg.eig(A)[0])
)

(4.499016357284714, array([0.40163425, 0.45887678, 0.35255653, 0.42177956, 0.57090098])) 
 4.499016357284721


## Inverse power iteration method with shifts

**Task 3.** Use the code for matrix inversion with the LU decomposition that you coded in the previous in-class assignment. (If you have not finished that part and do not have a working code, use `numpy.linalg.inv` instead.) Compute the inverse of the matrix that you produced in the previous exercise, and then use the power method that you coded to compute its eigenvalue. See if this application finds the smallest eigenvalue of the original matrix. (Do not forget to take the inverse of the eigenvalue that you found. Running the power method on the inverse matrix finds its largest eigenvalue -- which is the inverse of the smallest eigenvalue of the original matrix.)


In [584]:
def forsub(L,bs):
    n = bs.size
    xs = np.zeros(n)
    for i in range(n):
        val = 0.
        for j in range(i):
            val += L[i,j]*xs[j]
        xs[i] = (bs[i] - val)/L[i,i]
    return xs


def backsub(U,bs):
    n = bs.size
    xs = np.zeros(n)
    for i in reversed(range(n)):
        val = 0.
        for j in range(i+1,n):
            val += U[i,j]*xs[j]
        xs[i] = (bs[i] - val)/U[i,i]
    return xs

def ludec( A ):
    n = A.shape[0]
    U = np.copy(A)
    L = np.identity(n)

    for j in range(n-1):
        for i in range(j+1,n):
            coeff = U[i,j]/U[j,j]
            U[i,j:] -= coeff*U[j,j:]
            L[i,j] = coeff

    return L, U

def mat_inverse(A):
    n = A.shape[0]
    L, U = ludec(A)
    I = np.eye(n)
    inv_A = np.zeros((n, n))
    for i in range(n):
        Y = forsub(L, I[:, i])
        X = backsub(U, Y)
        inv_A[:, i] = X

    return inv_A

**Task 4.** Now that we understand the general idea, code the inverse power iteration method with an eigenvalue shift. The idea of this method is the following:

- Subtract the shift $\sigma$ from the original matrix $A$ and invert, i.e., let $B=(A-\sigma \mathbb{1})^{-1}$.
- Run the power iteration method on $B$, let the returned eigenvalue be $\kappa$.
- Then $\kappa$ is the largest eigenvalue of $B$ and $\kappa=1/(\lambda-\sigma)$, where $\lambda$ is some eigenvalue of $A$. This procedure effectively finds the smallest $\lambda-\sigma$.
- Shift back by adding $\sigma$ to $1/\kappa$ ($=\lambda-\sigma$) and you get $\lambda$ -- one of the eigenvalues of the original matrix $A$ which is close to the input shift $\sigma$.
- Recall that together with $\lambda$ you also get the eigenvector that corresponds to that eigenvalue from the power iteration method.

Essentially, the outlined method allows for finding eigenvectors for known eigenvalues and we use this idea in the next task. If you coded the inverse power iteration method correctly, the final vector is normalized to have unit length.

Test your code by running it on the same matrix you created above and providing a value for the shift that is close to one of the eigenvalues (that you know from using `numpy.linalg.eig` above).


In [585]:
# inverse power method: given matrix A and shift sigma find the eigenvalue closest
# to sigma and its corresponding eigenvector (the shifted matrix A is inverted inside
# this subroutine, and the power iteration method is run on the inverse)
# Input:
# A -- (square) matrix
# sigma -- shift
# max_iter -- maximum number of iterations
# Output:
# eigenvalue of A close to the shift sigma, its eigenvector
def invpower_method( A, sigma=0, max_iter=100 ):
    
    # read off dimensions
    n = A.shape[0]
    # prepare the inverse for the power iteration method
    B = mat_inverse( A - sigma*np.eye( n ) )
    kappa, Bvec = power_method(B) #A and B have the same eigenvectors
    eigenvalue = sigma + 1/kappa

    return eigenvalue, Bvec

In [586]:
# YOUR CODE HERE
eigenvals, eigenvecs = np.linalg.eig(A)
eigenval = eigenvals[2]

for eigenval in eigenvals:
    result_eigenval, eigenvec = invpower_method(A, eigenval)
    print(eigenvec)


[0.40163425 0.45887678 0.35255653 0.42177956 0.57090098]
[ 0.19049617 -0.38587861  0.62923828 -0.60360849  0.23350566]
[ 0.13563534  0.76218267  0.03786448 -0.53501774 -0.33615908]
[ 0.88537464 -0.24145961 -0.30864498  0.01546805 -0.24961576]
[ 0.01076773 -0.03611968  0.61892459  0.41385451 -0.66651049]


In [587]:
eigenvecs.T

array([[-0.40163425, -0.45887678, -0.35255653, -0.42177956, -0.57090098],
       [-0.19049617,  0.38587861, -0.62923828,  0.60360849, -0.23350566],
       [ 0.13563534,  0.76218267,  0.03786448, -0.53501774, -0.33615908],
       [-0.88537464,  0.24145961,  0.30864498, -0.01546805,  0.24961576],
       [-0.01076773,  0.03611968, -0.61892459, -0.41385451,  0.66651049]])

## QR decomposition

**Task 5.** Complete the function performing QR decomposition below using the code from the book of Gezerlis, chapter 4 (code 4.8 in the 2023 edition). Test it by comparing to `np.linalg.qr` (read the NumPy documentation, use `complete` for the `mode`).

**Note:** You expect the magnitudes to match but the signs may be different. (We are following the convention of Gezerlis where the diagonal elements of $R$ are taken as always positive. In this case the QR decomposition is unique.)

In [588]:
# QR decomposition of a (square) matrix A = QR
# Input:
# A -- (square) matrix
# Output:
# Q, R

def qrdec(A):
    n = A.shape[0]
    Ap = np.copy(A)
    Q = np.zeros((n,n))
    R = np.zeros((n,n))
    for j in range(n):
        for i in range(j):
            R[i,j] = Q[:,i]@A[:,j]
            Ap[:,j] -= R[i,j]*Q[:,i]

        R[j,j] = np.linalg.norm(Ap[:,j])
        Q[:,j] = Ap[:,j]/R[j,j]
    return Q, R


In [589]:
# YOUR CODE HERE
Q, R = np.linalg.qr(A, mode='complete')
myQ, myR = qrdec(A)
assert np.sum(abs(Q)-abs(myQ)) < 1e-14
assert np.sum(abs(R)-abs(myR)) < 1e-14

## QR method for finding eigenvalues


**Task 6.** Complete the function below that uses QR decomposition to find the eigenvalues of the (symmetric) matrix that you produced at the beginning of the notebook. Output the current estimate of the eigenvalues during each iteration, so you can observe the convergence of the method. (This can be controlled by a logical variable in the arguments `print_iter` so you can easily switch this printing functionality on and off.)


In [ ]:
# find all eigenvalues of a matrix by iterating the QR decomposition
# Input:
# A -- (square) matrix
# max_iter -- maximum number of iterations
# print_iter -- if True, print the current state of the eigenvalues
# Output:
# array with the eigenvalues of A
def eigenvalues_with_QR( A, max_iter=100, print_iter=False ):

    # store a copy so that the original matrix is not disturbed
    Ak = A.copy()
    for i in range(max_iter):
        eigenvals = np.sort(np.diagonal(Ak))

        Q, R = qrdec(Ak)
        Ak = R@Q # also Q.T@Ak@Q
        
        if all(eigenvals == np.sort(np.diagonal(Ak))): #convergence to machine accuracy
            return 0, eigenvals
        
        if print_iter:
            print(i, eigenvals)

    return 1, eigenvals # did not reach machine accuracy

In [591]:
eigenvalues_with_QR(A, max_iter=1000, print_iter=True)

0 [0.28077388 0.37252042 0.4089045  0.83404401 1.7527783 ]
1 [-1.00023026 -0.47622971  0.29105743  0.50873947  4.32568418]
2 [-1.29220332 -0.53800802  0.21192556  0.77927523  4.48803168]
3 [-1.33525589 -0.59423662  0.21497117  0.86548477  4.49805769]
4 [-1.34461931 -0.62816452  0.21595022  0.906924    4.49893073]
5 [-1.347023   -0.64728509  0.21608776  0.92823277  4.49900868]
6 [-1.34771579 -0.65741786  0.21610401  0.93903508  4.49901567]
7 [-1.34794117 -0.66260329  0.21610581  0.94444348  4.4990163 ]
8 [-1.34802436 -0.66521075  0.216106    0.94713387  4.49901635]
9 [-1.34805864 -0.66651061  0.21610602  0.94846799  4.49901636]
10 [-1.34807395 -0.66715587  0.21610602  0.94912855  4.49901636]
11 [-1.34808112 -0.66747551  0.21610602  0.94945536  4.49901636]
12 [-1.34808458 -0.66763368  0.21610602  0.949617    4.49901636]
13 [-1.34808627 -0.66771192  0.21610602  0.94969692  4.49901636]
14 [-1.3480871  -0.6677506   0.21610602  0.94973644  4.49901636]
15 [-1.34808752 -0.66776973  0.21610602 

(0, array([-1.34808792, -0.66778843,  0.21610602,  0.94977508,  4.49901636]))

## Computing eigenvectors for known eigenvalues

**Task 7.** Now there is everything in place to compute all eigenvectors of the symmetric matrix that you produced at the beginning of this notebook. First, compute the eigenvalues with the QR-based method that you coded in the previous exercise. Then loop over the eigenvalues, use them as shifts for the inverse power iteration method (to avoid divisions by zero, add $10^{-7}$ to the shift) to get the corresponding eigenvectors. Print the eigenvectors and compare with the result of `numpy.linalg.eig`. Remember that while the normalization in your code should be the same as in NumPy (i.e. the eigenvectors are normalized to have unit length), the overall sign is arbitrary. Thus, while you expect your results match the NumPy results in magnitude, they may have flipped signs.


In [592]:
success, eigenvals = eigenvalues_with_QR(A)

eigenvecs = []
for eigenval in eigenvals:
    r_eigenval, eigenvec = invpower_method(A, eigenval+1e-7)
    eigenvecs.append(eigenvec)

linalg_eigenval, linalg_eigenvec = np.linalg.eig(A)

qr_vals, qr_vecs = zip(*sorted(zip(eigenvals, np.abs(eigenvecs))))
lialg_vals, linalg_vecs = zip(*sorted(zip(linalg_eigenval, np.abs(linalg_eigenvec.T))))

assert np.all(np.isclose(qr_vecs, linalg_vecs, rtol=1e-14))

&#169; Copyright 2025,  Michigan State University Board of Trustees